# Libraries

In [1]:
import pyodbc
import pandas as pd
from openpyxl import load_workbook

# Database settings

In [2]:
# Connect to the database
server = 'DESKTOP-HTHQ40O\SQLEXPRESS'
database = 'SouthAfrica_CrimeStats'
username = 'cnoltehj'
password = 'HeidiJoy@1_2024'


# Validate Policestation and Exclude rows

In [53]:
# Load the Excel file
file_path_q1_2019_2023 = "./Dataset_CSV_Files/2023_2024/2023-2024-1st_Quarter_WEB.xlsx"

file_path_q2_2019_2023 = "./Dataset_CSV_Files/2023_2024/2023-2024_-_2nd_Quarter_WEB.xlsx"

file_path_q3_2019_2023 = "./Dataset_CSV_Files/2023_2024/2023-2024_-_3rd_Quarter_WEB.xlsx"

file_path_q4_2019_2023 = "./Dataset_CSV_Files/2023_2024/2023-2024_-_4th_Quarter_WEB.xlsx" 

#wb = load_workbook(file_path_q1_2019_2023, data_only=True)

#wb = load_workbook(file_path_q2_2019_2023, data_only=True)

#wb = load_workbook(file_path_q3_2019_2023, data_only=True)

wb = load_workbook(file_path_q4_2019_2023, data_only=True)

# Access the desired sheet
# Q 2,3,4
sheet = wb["Crime stats per component"]

# Quarter
#sheet = wb["Crime stats per component"]

# Check the value of cell Q8
cell_l8 = sheet["L8"].value

# Check the value of cell Q9
cell_l9 = sheet["L9"].value

# Q 2,3,4
excluded_rows = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 23, 28, 36, 40, 47, 52, 54, 55, 61, 62]
excluded_columns = [1, 2, sheet.max_column - 1, sheet.max_column]  # First 2 columns and last 2 columns

print("Province : ",cell_l8)
print("Police Station : ",cell_l9)

    # Iterate over all rows except for the excluded ones
for row_number in range(2, sheet.max_row + 1):
    if row_number not in excluded_rows:
            # Retrieve row values and exclude specified columns
        row_values = [cell.value for col_num, cell in enumerate(sheet[row_number]) if col_num + 1 not in excluded_columns]

            # Print the row values as a single dataset
        print(row_values)



Province :  Western Cape
Police Station :  Atlantis
['Murder', 17, 13, 18, 4, 10, 6, '6 counts higher']
['Sexual offences', 19, 19, 29, 26, 35, 9, '9 counts higher']
['Attempted murder', 14, 19, 10, 12, 12, 0, '0 count diff']
['Assault with the intent to inflict grievous bodily harm', 73, 40, 53, 73, 49, -24, '24 counts lower']
['Common assault', 200, 190, 166, 212, 263, 51, 0.24056603773584895]
['Common robbery', 38, 27, 19, 29, 35, 6, '6 counts higher']
['Robbery with aggravating circumstances', 36, 26, 26, 31, 37, 6, '6 counts higher']
['Contact crime (Crimes against the person)', 397, 334, 321, 387, 441, 54, 0.13953488372093026]
['Rape', 8, 13, 20, 15, 14, -1, '1 count lower']
['Sexual assault', 7, 4, 7, 8, 16, 8, '8 counts higher']
['Attempted sexual offences', 2, 0, 2, 3, 4, 1, '1 count higher']
['Contact sexual offences', 2, 2, 0, 0, 1, 1, '1 count higher']
['Carjacking', 0, 0, 0, 3, 0, -3, '3 counts lower']
['Robbery at residential premises', 5, 6, 1, 2, 5, 3, '3 counts higher'

# Insert Stats 2019 -2023 into Database

In [54]:
# Define the province parameter
selected_province = "ZA.WC"
selected_policestation = "AT04WC"
selected_quarter = 4

cnxn = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
cursor = cnxn.cursor()

# Iterate over all rows except excluded ones
for row_number in range(3, sheet.max_row + 1):  # Start from the third row
    
    if row_number not in excluded_rows:
        # Retrieve row values and exclude specified columns
        row_values = [cell.value for col_num, cell in enumerate(sheet[row_number]) if col_num + 1 not in excluded_columns]
        
        # Ensure row_values has enough elements before accessing
        #if len(row_values) > 1:
            # Convert the elements to int if they represent numeric values
        for i in range(1, 1):
            try:
                    row_values[i] = int(row_values[i])
            except (ValueError, IndexError):
                    # Handle the case where the value cannot be converted to int or index error occurs
                pass  # Handle this case as per your requirement
                
            # Inside the loop DB
        try:
                cursor.execute("EXEC sp_Add_QuarterlyCrimeStats ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?",
                   (str(row_values[0]).strip(), row_values[1], row_values[2], row_values[3], row_values[4], row_values[5], row_values[6], str(row_values[7]).strip(), str(selected_province).strip(), str(selected_policestation).strip(), int(selected_quarter)))
                print('Input 1')
                
                # Commit the changes to the database
                cnxn.commit()

                # Print row_values for debugging
                print("Row values:", row_values)

        except pyodbc.Error as e:
                print("Error executing SQL query:", e)


# Close the cursor and connection
cursor.close()
cnxn.close()

Input 1
Row values: ['Murder', 17, 13, 18, 4, 10, 6, '6 counts higher']
Input 1
Row values: ['Sexual offences', 19, 19, 29, 26, 35, 9, '9 counts higher']
Input 1
Row values: ['Attempted murder', 14, 19, 10, 12, 12, 0, '0 count diff']
Input 1
Row values: ['Assault with the intent to inflict grievous bodily harm', 73, 40, 53, 73, 49, -24, '24 counts lower']
Input 1
Row values: ['Common assault', 200, 190, 166, 212, 263, 51, 0.24056603773584895]
Input 1
Row values: ['Common robbery', 38, 27, 19, 29, 35, 6, '6 counts higher']
Input 1
Row values: ['Robbery with aggravating circumstances', 36, 26, 26, 31, 37, 6, '6 counts higher']
Input 1
Row values: ['Contact crime (Crimes against the person)', 397, 334, 321, 387, 441, 54, 0.13953488372093026]
Input 1
Row values: ['Rape', 8, 13, 20, 15, 14, -1, '1 count lower']
Input 1
Row values: ['Sexual assault', 7, 4, 7, 8, 16, 8, '8 counts higher']
Input 1
Row values: ['Attempted sexual offences', 2, 0, 2, 3, 4, 1, '1 count higher']
Input 1
Row values:

# Get Extra years for 2016 sheets and exclude rows

In [29]:
# Load the Excel file

file_path_q1_2016_2018 = "./Dataset_CSV_Files/2020_2021/first_quarter 2020_2021_crime_statistics.xlsx"

file_path_q2_2016_2018 = "./Dataset_CSV_Files/2020_2021/crime_statistics_july_september_2020_21.xlsx"

file_path_q3_2016_2018 = "./Dataset_CSV_Files/2020_2021/crime_statistics_third_qaurter 2020_2021.xlsx"

file_path_q4_2016_2018 = "./Dataset_CSV_Files/2020_2021/crime_statistics_fourth_qaurter 2020_2021_current.xlsx"

wb = load_workbook(file_path_q4_2016_2018, data_only=True)

# Access the desired sheet
# 1 Quarter
#sheet = wb["Dropdown tables stations"]

# 2 Quarter
#sheet = wb["Stations per province"]

# 3 Quarter
# sheet = wb["Quarterly stations p province"]

# 3 Quarter
sheet = wb["Quarterly stations per province"]

# Check the value of cell Q8
cell_j9 = sheet["J9"].value

# Check the value of cell Q9
cell_j10 = sheet["J10"].value

# 1 Quarter
#excluded_rows = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 22, 28, 29, 36, 40, 46, 53]

# 2,3 Quarter
excluded_rows = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 22, 28, 29, 36, 40, 47, 53]


excluded_columns = [1, 2, sheet.max_column - 1, sheet.max_column]  # First 2 columns and last 2 columns

print("Province : ",cell_j9)
print("Police Station : ",cell_j10)

    # Iterate over all rows except for the excluded ones
for row_number in range(2, sheet.max_row + 1):
    if row_number not in excluded_rows:
            # Retrieve row values and exclude specified columns
        row_values = [cell.value for col_num, cell in enumerate(sheet[row_number]) if col_num + 1 not in excluded_columns]

            # Print the row values as a single dataset
        print(row_values)
     

Province :  WESTERN_CAPE
Police Station :  ATLANTIS
['Murder', 12, 16, 7, 17, 13, -4, -0.23529411764705888, None, None, None, 'ALEXANDRIA', 'BAYSWATER', 'ATTERIDGEVILLE', 'BAYVIEW', 'BOLOBEDU', 'BALFOUR TVL', 'BETHANIE', 'BATLHAROS', 'BARRYDALE', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
['Sexual Offences', 17, 20, 22, 19, 19, 0, 0, None, None, None, 'ALGOAPARK', 'BETHLEHEM', 'BEDFORDVIEW', 'BELLAIR', 'BOTLOKWA', 'BARBERTON', 'BIESIESVLEI', 'BELMONT', 'BEAUFORT WEST', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
['Attempted murder', 14, 19, 8, 14, 19, 5, 0.3571428571428572, None, None, None, 'ALICE', 'BETHULIE', 'BEKKERSDAL', 'BEREA', 'BULGERIVIER', 'BELFAST', 'BLOEMHOF', 'BOETSAP', 'BELHAR', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
['Assault with the intent to inflict grievo

# Insert Extra years into database

In [30]:
import pyodbc
from openpyxl import load_workbook

# Connect to the database
server = 'DESKTOP-HTHQ40O\SQLEXPRESS'
database = 'SouthAfrica_CrimeStats'
username = 'cnoltehj'
password = 'HeidiJoy@1_2024'
cnxn = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
cursor = cnxn.cursor()

# Define the province parameter
selected_province = "ZA.WC"
selected_policestation = "AT04WC"
selected_quarter = 4

# Iterate over all rows except excluded ones
for row_number in range(3, sheet.max_row + 1):  # Start from the third row
    
    if row_number not in excluded_rows:
        # Retrieve row values and exclude specified columns
        row_values = [cell.value for col_num, cell in enumerate(sheet[row_number]) if col_num + 1 not in excluded_columns]
        
        # Ensure row_values has enough elements before accessing
        #if len(row_values) > 1:
            # Convert the elements to int if they represent numeric values
        for i in range(1, 1):
            try:
                    row_values[i] = int(row_values[i])
            except (ValueError, IndexError):
                    # Handle the case where the value cannot be converted to int or index error occurs
                pass  # Handle this case as per your requirement
                
            # Inside the loop
        try:
                cursor.execute("EXEC sp_Add_QuarterlyCrimeStats_PreviousYears ?, ?, ?, ?, ?, ?, ?",
                   (str(row_values[0]).strip(), row_values[1], row_values[2], row_values[3], str(selected_province).strip(), str(selected_policestation).strip(), int(selected_quarter)))
                

                # Commit the changes to the database
                cnxn.commit()

                # Print row_values for debugging
                print("Row values:", row_values)

        except pyodbc.Error as e:
                print("Error executing SQL query:", e)


# Close the cursor and connection
cursor.close()
cnxn.close()

Row values: ['Murder', 12, 16, 7, 17, 13, -4, -0.23529411764705888, None, None, None, 'ALEXANDRIA', 'BAYSWATER', 'ATTERIDGEVILLE', 'BAYVIEW', 'BOLOBEDU', 'BALFOUR TVL', 'BETHANIE', 'BATLHAROS', 'BARRYDALE', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Row values: ['Sexual Offences', 17, 20, 22, 19, 19, 0, 0, None, None, None, 'ALGOAPARK', 'BETHLEHEM', 'BEDFORDVIEW', 'BELLAIR', 'BOTLOKWA', 'BARBERTON', 'BIESIESVLEI', 'BELMONT', 'BEAUFORT WEST', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Row values: ['Attempted murder', 14, 19, 8, 14, 19, 5, 0.3571428571428572, None, None, None, 'ALICE', 'BETHULIE', 'BEKKERSDAL', 'BEREA', 'BULGERIVIER', 'BELFAST', 'BLOEMHOF', 'BOETSAP', 'BELHAR', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Row values: ['Assault with the intent to inflict grievous b